In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.writer import save_offset_to_minio, write_to_parquet, write_to_postgres
from utils.reader import load_offset_from_minio, read_from_parquet
from utils.fetch_data import fetch_kafka_data
from utils.transform_data import transform_data
from clients.spark_client import create_spark_session
from db_utils.check_gp import create_db_schema_table
from clients.postgres_client import get_postgres_properties
from config import MINIO_RAW_PATH, MINIO_PROCESSED_PATH, DB_TABLE

In [3]:
create_db_schema_table()

База данных weather уже существует.
Схема weather уже существует.
Схема и таблица успешно созданы.


!!! RESET OFFSET VALUE

In [10]:
save_offset_to_minio(-1)


In [4]:
offset = load_offset_from_minio()
print(offset)

29


In [7]:
spark = create_spark_session()


In [6]:
last_offset = fetch_kafka_data(spark, MINIO_RAW_PATH, offset)
save_offset_to_minio(last_offset)
print(last_offset)

[{'country': 'RU', 'city': 'Moscow', 'timestamp': '2025-07-31T16:16:15', 'temperature': 301.48, 'feels_like': 301.43, 'pressure': 1009, 'humidity': 44, 'weather_main': 'Clouds', 'weather_description': 'broken clouds', 'wind_speed': 4.71, 'wind_deg': 119, 'clouds': 63}, {'country': 'RU', 'city': 'Moscow', 'timestamp': '2025-07-31T16:29:21', 'temperature': 301.2, 'feels_like': 301.23, 'pressure': 1009, 'humidity': 45, 'weather_main': 'Clouds', 'weather_description': 'broken clouds', 'wind_speed': 4.71, 'wind_deg': 119, 'clouds': 63}, {'country': 'RU', 'city': 'Moscow', 'timestamp': '2025-07-31T16:29:21', 'temperature': 301.2, 'feels_like': 301.23, 'pressure': 1009, 'humidity': 45, 'weather_main': 'Clouds', 'weather_description': 'broken clouds', 'wind_speed': 4.71, 'wind_deg': 119, 'clouds': 63}, {'country': 'RU', 'city': 'Moscow', 'timestamp': '2025-07-31T16:45:32', 'temperature': 300.41, 'feels_like': 300.56, 'pressure': 1009, 'humidity': 46, 'weather_main': 'Clouds', 'weather_descript

25/08/06 14:07:16 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/08/06 14:07:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


43


In [8]:
df_transform = transform_data(spark, MINIO_RAW_PATH, MINIO_PROCESSED_PATH)
write_to_parquet(spark, df_transform, MINIO_PROCESSED_PATH)


25/08/06 14:10:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [12]:
df_transform_pq = read_from_parquet(spark, MINIO_PROCESSED_PATH)
properties = get_postgres_properties()
write_to_postgres(df_transform_pq, DB_TABLE, properties)